### **1 - Importações**

---

#### **1.1 - Importando Bibliotecas Comuns**

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import joblib
import lightgbm as lgb


---
#### **1.2 - Importando o conjunto de dados**

In [2]:
# Define o caminho dos dados e transforma em dataframe
data_path = "../data/teste_indicium_precificacao.csv"
df = pd.read_csv(data_path)

---

### **2 - Limpeza e tratamento dos dados**

---

#### **2.1 - Remoção de variáveis desnecessárias**

As variáveis removidas serão:
- id
    - Variável que contém um identificador único do anúncio.
- nome
    - Variável que contém o nome único do anúncio.
- host_id
    - Variável que contém um identificador único do anfitrião.
- host_name
    - Variável host_name que contém o nome do anfitrião.
- ultima_review
    - Variável que contém a data da ultima avaliação.
- latitude
    - Variável que contém a latitude, apresentou alta correlação com bairro e latitude.
- longitude
    - Variável que contém a longitude, apresentou alta correlação com bairro e longitude.

In [3]:
# Remove colunas desnecessárias
df.drop(['id', 'nome', 'host_id', 'host_name', 'ultima_review', 'latitude', 'longitude'], axis=1, inplace=True)

---

#### **2.2 - Verificação e remoção de ausentes**

In [4]:
# Verifica os valores ausentes
possui_ausente = (df.isnull().sum() / df.shape[0]) * 100
possui_ausente = possui_ausente[possui_ausente > 0].sort_values(ascending=False)

print(possui_ausente)

reviews_por_mes    20.55876
dtype: float64


Com base no analisado durante a EDA, podemos inferir que a presença valores nulos se deve ao fato de que alguns imóveis nunca foram alugados ou ainda não estiveram disponíveis. Notamos isso na relação de zeros e nulos das colunas reviews_por_mes e ultima_review (já removida). 

Não seria adequado remover linhas inteiras ou imputar pelas medidas de tendência central, optou-se então pela substituição por 0.

In [5]:
# Substitui NaN por 0
df['reviews_por_mes']=df['reviews_por_mes'].fillna(0)

---

#### **2.3 - Verificação e remoção de outliers**

In [6]:
def identificar_outliers(df, coluna):
    """
    Identifica outliers em uma coluna usando a regra do IQR.
    Retorna uma máscara booleana indicando os outliers.
    """
    Q1 = df[coluna].quantile(0.25)  # Primeiro quartil (25%)
    Q3 = df[coluna].quantile(0.75)  # Terceiro quartil (75%)
    IQR = Q3 - Q1                   # Intervalo interquartil

    # Limites inferior e superior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Identifica outliers
    outliers = (df[coluna] < limite_inferior) | (df[coluna] > limite_superior)
    return outliers

# Lista das colunas para verificar outliers
colunas_analisar = [
    'minimo_noites', 'numero_de_reviews', 'reviews_por_mes',
    'calculado_host_listings_count', 'disponibilidade_365', 'price'
]

# Verifica outliers em cada coluna
for coluna in colunas_analisar:
    outliers = identificar_outliers(df, coluna)
    print(f"Outliers na coluna '{coluna}': {outliers.sum()} ({(outliers.sum() / len(df)) * 100:.2f}%)")

Outliers na coluna 'minimo_noites': 6607 (13.51%)
Outliers na coluna 'numero_de_reviews': 6021 (12.31%)
Outliers na coluna 'reviews_por_mes': 3312 (6.77%)
Outliers na coluna 'calculado_host_listings_count': 7080 (14.48%)
Outliers na coluna 'disponibilidade_365': 0 (0.00%)
Outliers na coluna 'price': 2972 (6.08%)


In [7]:
def tratar_outliers(df, coluna):
    """
    Trata outliers em uma coluna usando a técnica de cap.
    Retorna o DataFrame com os outliers tratados.
    """
    Q1 = df[coluna].quantile(0.25)  # Primeiro quartil (25%)
    Q3 = df[coluna].quantile(0.75)  # Terceiro quartil (75%)
    IQR = Q3 - Q1                   # Intervalo interquartil

    # Limites inferior e superior
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR

    # Aplica o cap
    df[coluna] = df[coluna].clip(lower=limite_inferior, upper=limite_superior)
    return df

# Aplica o tratamento de outliers em cada coluna
for coluna in colunas_analisar:
    df = tratar_outliers(df, coluna)

# Verifica se os outliers foram tratados
for coluna in colunas_analisar:
    outliers = identificar_outliers(df, coluna)
    print(f"Outliers na coluna '{coluna}' após tratamento: {outliers.sum()} ({(outliers.sum() / len(df)) * 100:.2f}%)")

Outliers na coluna 'minimo_noites' após tratamento: 0 (0.00%)
Outliers na coluna 'numero_de_reviews' após tratamento: 0 (0.00%)
Outliers na coluna 'reviews_por_mes' após tratamento: 0 (0.00%)
Outliers na coluna 'calculado_host_listings_count' após tratamento: 0 (0.00%)
Outliers na coluna 'disponibilidade_365' após tratamento: 0 (0.00%)
Outliers na coluna 'price' após tratamento: 0 (0.00%)


---

#### **2.4 - Filtragem de bairros com alta taxa de registros**

Ao rodar os modelos pela primeira vez, foi verificado que a variável bairro apresentou um comportamento não esperado, portanto, aqui serão filtrados os bairros com uma quantidade n de registros, a fim de possibilitar uma avaliação mais fiel dos dados.

In [8]:
# Verifica os bairros com um n de registros
unique_bairros = df['bairro'].value_counts()
bairros_com_poucos_registro = unique_bairros[unique_bairros < 200].index
bairros_com_poucos_registro

Index(['Jackson Heights', 'East Elmhurst', 'Boerum Hill', 'Tribeca',
       'Kensington', 'Sheepshead Bay', 'Windsor Terrace', 'Brooklyn Heights',
       'Canarsie', 'Forest Hills',
       ...
       'Silver Lake', 'West Farms', 'Bay Terrace, Staten Island',
       'Howland Hook', 'Woodrow', 'Richmondtown', 'Fort Wadsworth', 'New Dorp',
       'Rossville', 'Willowbrook'],
      dtype='object', name='bairro', length=173)

In [9]:
# Verifica se o bairro está nos registros e modifica o df
df = df[~df['bairro'].isin(bairros_com_poucos_registro)] 
df['bairro'].count()

np.int64(42276)

In [10]:
# Tranforma os tipos de colunas, se necessário.
df['bairro_group'] = df['bairro_group'].astype("category")
df['bairro'] = df['bairro'].astype("category")
df['room_type'] = df['room_type'].astype("category")

In [11]:
df.head()

,bairro_group,bairro,room_type,price,minimo_noites,numero_de_reviews,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,Manhattan,Midtown,Entire home/apt,225,1,45.0,0.38,2.0,355
1,Manhattan,Harlem,Private room,150,3,0.0,0.00,1.0,365
2,Brooklyn,Clinton Hill,Entire home/apt,89,1,58.5,3.89,1.0,194
3,Manhattan,East Harlem,Entire home/apt,80,10,9.0,0.10,1.0,0
4,Manhattan,Murray Hill,Entire home/apt,200,3,58.5,0.59,1.0,129


---

### **3 - Divisão dos dados**

In [12]:
# Define os dados de entrada X e saída y:
X = df[['bairro_group', 'bairro', 'room_type', 'minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']]
y = df['price']

# Divide os dados em conjunto de treinamento e teste:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

---

### 4 - **Transformação de variáveis**

In [13]:
# Define quais colunas devem ser codificadas one-hot Encoder
categorical_features = ['bairro_group', 'bairro', 'room_type']

# Define quais colunas devem ser padronizadas
numeric_features = ['minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']

In [14]:
# Cria um ColumnTransformer para aplicar diferentes transformações a diferentes colunas
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(), categorical_features),  # Aplica OneHotEncoder às colunas categóricas
        ('numeric', StandardScaler(), numeric_features)  # Aplica StandardScaler às colunas numéricas
    ],
    remainder='passthrough'
)

---

### **5 - Testando modelos**

---

#### **5.1 - Regressão Linear**

Um Pipeline do scikit-learn é usado para criar uma sequência de etapas que vão ser executadas uma após a outra. 

Sua primeira etapa é a aplicação de transformações nos dados.

A segunda etapa do pipeline é a aplicação do modelo escolhido. 

Iniciaremos com regressão linear, um dos algoritmos mais simples para prever um valor contínuo com base em variáveis independentes.

In [15]:
# Cria um pipeline com ColumnTransformer e um modelo de regressão linear
linear_model = Pipeline([
    ('preprocessor', preprocessor),  # Aplicar o ColumnTransformer
    ('regressor', LinearRegression())  # Modelo de regressão linear
])

In [16]:
# Treina o modelo
linear_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['bairro_group', 'bairro',
                                                   'room_type']),
                                                 ('numeric', StandardScaler(),
                                                  ['minimo_noites',
                                                   'numero_de_reviews',
                                                   'reviews_por_mes',
                                                   'calculado_host_listings_count',
                                                   'disponibilidade_365'])])),
                ('regressor', LinearRegression())])

In [17]:
# Faz previsões
linear_model_predictions = linear_model.predict(X_test)

In [30]:
# Calcula o MSE
mse = mean_squared_error(y_test, linear_model_predictions)

# Calcula o RMSE usando numpy.sqrt()
rmse_lr = np.sqrt(mse)

# Imprime o resultado
print(f'RMSE: {rmse_lr}')

RMSE: 59.04845849834057


---

#### **5.2 - SVR**

O SVR (Support Vector Regression) é uma técnica de aprendizado supervisionado voltada para problemas de regressão.

O Pipeline funciona da mesma maneira, só aplicamos outro modelo.

In [26]:
# Cria um pipeline com ColumnTransformer e um modelo SVR
svr_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', SVR(kernel = 'rbf'))
])

In [27]:
# Treina o modelo
svr_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['bairro_group', 'bairro',
                                                   'room_type']),
                                                 ('numeric', StandardScaler(),
                                                  ['minimo_noites',
                                                   'numero_de_reviews',
                                                   'reviews_por_mes',
                                                   'calculado_host_listings_count',
                                                   'disponibilidade_365'])])),
                ('regressor', SVR())])

In [28]:
# Faz previsões
svr_model_predictions = svr_model.predict(X_test)

In [31]:
# Calcula o MSE
mse = mean_squared_error(y_test, svr_model_predictions)

# Calcula o RMSE usando numpy.sqrt()
rmse_svr = np.sqrt(mse)

# Imprime o resultado
print(f'RMSE do SVR: {rmse_svr}')

RMSE do SVR: 61.24690728916862


---

### **5.3 - Random Forest**

O Random Forest é um modelo *ensemble* (combina múltiplos modelos para melhorar a precisão). A ideia é que, quando você combina múltiplos modelos, a precisão geral tende a melhorar, já que os erros de um modelo podem ser compensados pelos outros.

In [32]:
# Cria um pipeline com ColumnTransformer e o modelo de Random Forest
random_forest_n20_model = Pipeline([
    ('preprocessor', preprocessor), 
    ('regressor', RandomForestRegressor(n_estimators = 20, random_state = 0))
])

In [33]:
# Treina o modelo
random_forest_n20_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['bairro_group', 'bairro',
                                                   'room_type']),
                                                 ('numeric', StandardScaler(),
                                                  ['minimo_noites',
                                                   'numero_de_reviews',
                                                   'reviews_por_mes',
                                                   'calculado_host_listings_count',
                                                   'disponibilidade_365'])])),
                ('regressor',
                 RandomForestRegressor(n_estimators=20, random_state=0))])

In [35]:
# Faz previsões
random_forest_n20_model_predictions = random_forest_n20_model.predict(X_test)

In [36]:
# Calcula o RMSE para o modelo Random Forest
rmse_rf = np.sqrt(mean_squared_error(y_test, random_forest_n20_model_predictions))
rmse_rf

np.float64(58.729608482512965)

---

### 5.4 - **Gradient Boosting**

Semelhante ao Random Forest. 

A ideia central do boosting é que, ao treinar sucessivamente novos modelos para corrigir os erros dos modelos anteriores, o desempenho geral melhora.

In [37]:
# Cria o pipeline com Gradient Boosting
gradient_boosting_model = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', GradientBoostingRegressor(random_state=42))
])

# Treina o modelo
gradient_boosting_model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['bairro_group', 'bairro',
                                                   'room_type']),
                                                 ('numeric', StandardScaler(),
                                                  ['minimo_noites',
                                                   'numero_de_reviews',
                                                   'reviews_por_mes',
                                                   'calculado_host_listings_count',
                                                   'disponibilidade_365'])])),
                ('regressor', GradientBoostingRegressor(random_state=42))])

In [41]:
# Faz previsões
previsao_gb = gradient_boosting_model.predict(X_test)

In [42]:
# Calcula o RMSE para o modelo Gradient Boosting
rmse_gb = np.sqrt(mean_squared_error(y_test, previsao_gb))
rmse_gb

np.float64(57.291051523848765)

---

### **5.5 - LightGBM**

LightGBM segue a ideia central do Gradient Boosting, onde múltiplos modelos fracos (geralmente, árvores de decisão simples) são combinados para formar um modelo forte e preciso.

In [43]:
# Cria o pipeline com LightGBM
lightgbm_model = Pipeline([
    ('preprocessor', preprocessor), 
    ('regressor', lgb.LGBMRegressor(random_state=42, force_row_wise=True),)
])

In [44]:
# Treina o modelo
lightgbm_model.fit(X_train, y_train)

[LightGBM] [Info] Total Bins 688
[LightGBM] [Info] Number of data points in the train set: 33820, number of used features: 59
[LightGBM] [Info] Start training from score 136.477203


c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical',
                                                  OneHotEncoder(),
                                                  ['bairro_group', 'bairro',
                                                   'room_type']),
                                                 ('numeric', StandardScaler(),
                                                  ['minimo_noites',
                                                   'numero_de_reviews',
                                                   'reviews_por_mes',
                                                   'calculado_host_listings_count',
                                                   'disponibilidade_365'])])),
                ('regressor',
                 LGBMRegressor(force_row_wise=True, random_state=42))])

In [45]:
# Faz previsões com o modelo LightGBM
lightgbm_predictions = lightgbm_model.predict(X_test)

# Calcula o RMSE para o modelo LightGBM
rmse_lgbm = np.sqrt(mean_squared_error(y_test, lightgbm_predictions))
print(f'O RMSE do modelo LightGBM foi: {rmse_lgbm}')

O RMSE do modelo LightGBM foi: 55.49245258262465


c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


---

### **6 - Comparação entre modelos**

In [47]:
# Compara os modelos
print(f'O resultado do modelo de Regressão Linear foi: RMSE = {rmse_lr}')
print(f'O resultado do modelo SVR foi: RMSE = {rmse_svr}')
print(f'O resultado do modelo Random Forest foi: RMSE = {rmse_rf}')
print(f'O resultado do modelo Gradient Boosting foi: RMSE = {rmse_gb}')
print(f'O resultado do modelo LightGBM foi: RMSE = {rmse_lgbm}')

O resultado do modelo de Regressão Linear foi: RMSE = 59.04845849834057
O resultado do modelo SVR foi: RMSE = 61.24690728916862
O resultado do modelo Random Forest foi: RMSE = 58.729608482512965
O resultado do modelo Gradient Boosting foi: RMSE = 57.291051523848765
O resultado do modelo LightGBM foi: RMSE = 55.49245258262465


O melhor modelo analisado foi o **LightGBM**, portanto, veremos o valor estimado do apartamento utilizando esse modelo.

Com o menor RMSE (55.49) ainda não podemos afirmar ter um bom resultado.

Após, para tentar melhorar a performance do mesmo, vamos testar outros parâmetros e ver como o modelo se comporta.

---

#### **6.1 - Adição dos dados de interesse**

In [48]:
def transform_to_X_test(data):
    required_keys = ['nome', 'room_type', 'bairro', 'bairro_group', 'minimo_noites', 
                     'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 
                     'disponibilidade_365']
    
    # Filtra os dados, pegando apenas as chaves necessárias
    data_filtered = {key: data.get(key) for key in required_keys}
    
    X_test = {
        'nome': [data_filtered.get('nome')],
        'room_type': [data_filtered.get('room_type')],
        'bairro': [data_filtered.get('bairro')],
        'bairro_group': [data_filtered.get('bairro_group')],
        'minimo_noites': [data_filtered.get('minimo_noites')],
        'numero_de_reviews': [data_filtered.get('numero_de_reviews')],
        'reviews_por_mes': [data_filtered.get('reviews_por_mes')],
        'calculado_host_listings_count': [data_filtered.get('calculado_host_listings_count')],
        'disponibilidade_365': [data_filtered.get('disponibilidade_365')]
    }
    return pd.DataFrame(X_test)

In [49]:
# Chama a função passando um dicionário de dados com as informações selecionadas.
teste = transform_to_X_test({'id': 2595,
 'nome': 'Skylit Midtown Castle',
 'host_id': 2845,
 'host_name': 'Jennifer',
 'bairro_group': 'Manhattan',
 'bairro': 'Midtown',
 'latitude': 40.75362,
 'longitude': -73.98377,
 'room_type': 'Entire home/apt',
 'price': 225,
 'minimo_noites': 1,
 'numero_de_reviews': 45,
 'ultima_review': '2019-05-21',
 'reviews_por_mes': 0.38,
 'calculado_host_listings_count': 2,
 'disponibilidade_365': 355})

---

### **7 - Previsão com base nos dados de interesse**

In [50]:
# Faz uma previsão do 'price' usando o LightGBM e os dados selecionados.
new_predictions = lightgbm_model.predict(teste)
print(new_predictions)

[247.60704342]


c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


A previsão, utilizando o melhor modelo ficaria em $247.60.

---

### **8 - Ajuste de Hiperparâmetros**
Ajustar os hiperparâmetros do modelo usando uma busca aleatória (RandomizedSearchCV) é uma maneira eficiente de encontrar as melhores configurações para o modelo, sem precisar testar todas as combinações possíveis (como ocorre na busca em grade).


In [51]:
# Definindo as colunas categóricas e numéricas
categorical_features = ['bairro_group', 'bairro', 'room_type']
numeric_features = ['minimo_noites', 'numero_de_reviews', 'reviews_por_mes', 
                    'calculado_host_listings_count', 'disponibilidade_365']

# Configurando o pré-processador
preprocessor = ColumnTransformer(
    transformers=[
        ('categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
        ('numeric', StandardScaler(), numeric_features)
    ],
    remainder='passthrough'
)

# Definindo o modelo LightGBM
lgb_model = lgb.LGBMRegressor(random_state=42)

# Parâmetros a serem testados na busca aleatória
param_dist = {
    'regressor__n_estimators': [100, 200, 300, 500, 1000],  
    'regressor__learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2],  
    'regressor__max_depth': [3, 5, 7, 9, -1],  
    'regressor__num_leaves': [31, 50, 100, 150],  
    'regressor__subsample': [0.6, 0.7, 0.8, 0.9, 1.0],  
    'regressor__colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],  
    'regressor__min_child_samples': [10, 20, 30, 50]  
}

# Configurando o pipeline com o pré-processador e o regressor
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', lgb_model)
])

# Configurando a busca aleatória
random_search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_dist,  
    n_iter=50, 
    scoring='neg_mean_squared_error',  
    cv=3,  
    verbose=1,  
    random_state=42,  
    n_jobs=-1
)

# Treinando o modelo com a busca aleatória
random_search.fit(X_train, y_train)

# Exibindo os melhores parâmetros encontrados
print("Melhores parâmetros encontrados: ", random_search.best_params_)

# Avaliando o modelo otimizado
best_lgb_model = random_search.best_estimator_
y_pred = best_lgb_model.predict(X_test)

# Faz previsões com o modelo LightGBM otimizado
lgb_optimized_predictions = best_lgb_model.predict(X_test)

# Calcula o RMSE para o modelo LightGBM otimizado
rmse_lgb_optimized = np.sqrt(mean_squared_error(y_test, lgb_optimized_predictions))
print(f'O RMSE do modelo LightGBM otimizado foi: {rmse_lgb_optimized}')

Fitting 3 folds for each of 50 candidates, totalling 150 fits


c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000391 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 688
[LightGBM] [Info] Number of data points in the train set: 33820, number of used features: 59
[LightGBM] [Info] Start training from score 136.477203
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Melhores parâmetros encontrados:  {'regressor__subsample': 0.6, 'regressor__num_leaves': 100, 'regressor__n_estimators': 1000, 'regressor__min_child_samples': 30, 'regressor__max_depth': -1, 'regressor__learning_rate': 0.01, 'regressor__colsample_bytree': 0.6}
O RMSE do modelo Ligh

c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Ajustar os hiperparâmetros não melhorou o modelo.

In [52]:
# Faz a previsão com o modelo otimizado
prediction = best_lgb_model.predict(teste)
print(prediction)

[261.47852646]


c:\Users\Ian\Documents\Análise de Dados\Projeto Indicium\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


A previsão, utilizando o modelo melhorado, ficaria em **$261.47**

---

### **9 - Exportando o modelo**

In [53]:
joblib.dump(best_lgb_model, '../models/LH_CD_Ian_Rodrigo_Stoltz.pkl')

['../models/LH_CD_Ian_Rodrigo_Stoltz.pkl']